# Лабораторная работа №6

**Сверточные нейронные сети**

---

**Впишите в эту ячейку ваши ФИО, группу**.

ФИО: Сильченко Алексей Евгеньевич 

Группа: 201-361

---


## Загрузка данных

В данной работе будет использоваться учебный датасет с изображениями персонажей из Симпсонов. Код для скачивания и распаковки приведен ниже, его требуется только выполнить и вo вкладке Files должна появиться папка `data`, а в ней папки `train` и `test`.

In [12]:
#Загрузка для windows
# Invoke-WebRequest -Uri http://labcolor.space/rgb-test.zip -OutFile rgb-test.zip
# Expand-Archive -Path rgb-test.zip -DestinationPath data -Force

# Invoke-WebRequest -Uri http://labcolor.space/rgb-train.zip -OutFile rgb-train.zip
# Expand-Archive -Path rgb-train.zip -DestinationPath data -Force

## Создание объекта Dataset

Так как изображения в датасете организованы по папкам, где имя папки является ярлыком для данных, то мы можем воспользоваться базовым классом `ImageFolder`.

Одним из параметров является `transform`, для которого необходимо скомпоновать преобразования для наших изображений. В pytorch для преобразований сейчас есть два набора функций V1 и V2 и рекомендуется использовать V2, хоть напротив многих функций указано состояние beta.

Для компоновки функции из модуля v2 используйте `Compose`. Вам понадобится обязательно:
* ToImage() - преобразование в `Image` (подкласс torch.Tensor)
* RandomVerticalFlip() - случайное отзеркаливание
* ToDtype(torch.float32, scale=True) - преобразование из int во float
* Normalize() - нормализация изображений по полученным средним и стандартным отклонениям.

По желанию:
* RandomRotation() - поворот на случайный угол в указанном диапазоне
* Можете попробовать и другие варианты преобразований. [Документация API V2](https://pytorch.org/vision/stable/transforms.html#v2-api-reference-recommended)

В качестве примера будет показана работа с созданием Dataset для получения статистик изображения. Вам же необходимо будет создать `transforms` для обучения и проверки. При обучении вы используете весь набор обязательных преобразований, при обучении вам требуется только преобразовать изображение к тензору с плавающей точкой и провести нормализацию.

### Получение статистик для нормализации

In [2]:
import torch
from torchvision.transforms import v2

transforms_stats = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
])

In [3]:
from torchvision.datasets import ImageFolder

stats_dataset = ImageFolder(root="./data/train", transform=transforms_stats)

In [4]:
# Собираем все изображения из датасета в список, где каждый элемент - это изображение.
imgs = [item[0] for item in stats_dataset]

# Преобразуем список изображений в тензор PyTorch для дальнейших вычислений,
# объединяя их в один тензор с добавлением нового измерения в начале,
# что соответствует размеру пакета (batch size).
imgs = torch.stack(imgs, dim=0).numpy()

# Вычисляем среднее значение пикселей канала красного, зеленого и синего по всем изображениям.
mean_r = imgs[:,0,:,:].mean()
mean_g = imgs[:,1,:,:].mean()
mean_b = imgs[:,2,:,:].mean()

# Выводим средние значения для каждого из цветовых каналов.
print(f"Means R, G, B: {mean_r,mean_g,mean_b}")

# Вычисляем стандартное отклонение пикселей канала красного, зеленого и синего по всем изображениям.
std_r = imgs[:,0,:,:].std()
std_g = imgs[:,1,:,:].std()
std_b = imgs[:,2,:,:].std()

# Выводим значения стандартных отклонений для каждого из цветовых каналов.
print(f"Std R, G, B: {std_r,std_g,std_b}")

Means R, G, B: (0.465204, 0.40914717, 0.35447764)
Std R, G, B: (0.2492806, 0.22941421, 0.2450912)


**Почему значения средних и стандартных отклонений мы получаем только для обучающей выборки?**

Ваш ответ: Нормализация данных(Средние и стандартные отклонения используются для нормализации данных обучающей выборки), предотвращение утечки данных, соответствие реальным условиям, универсальность модели.


---


Используйте выведенные выше значения средних и стандартных отклонений в качестве аргументов функции `Normalize`.

In [30]:
from torchvision.transforms import v2 as transforms

means = (0.465204, 0.40914717, 0.35447764) # Средние значения для каналов RGB
stds = (0.2492806, 0.22941421, 0.2450912) # Стандартные отклонения для каналов RGB

transforms_train = transforms.Compose([
    #transforms.Resize(256), # Изменение размера всех изображений до 256 пикселей по меньшей стороне
    #transforms.RandomCrop(224), # Случайное обрезание изображений до размера 224x224 пикселей
    transforms.RandomVerticalFlip(), # Случайное вертикальное отражение изображений
    transforms.ToImage(), # Преобразование в объекты изображения PIL
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean=means, std=stds),
])

transforms_test = transforms.Compose([
    #transforms.Resize(224), # Изменение размера всех изображений до 224 пикселей по меньшей стороне
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean=means, std=stds),
])

Теперь, когда есть необходимые `transforms` можно создать ImageFolder, указав в `root` путь до выборки и `transforms` в `transform`.

In [32]:

train_data_path = './data/train'
val_data_path = './data/test'

train_dataset = ImageFolder(root=train_data_path, transform=transforms_train)

val_dataset = ImageFolder(root=val_data_path, transform=transforms_test)

In [8]:
mean = [0.465204, 0.40914717, 0.35447764]
std = [0.2492806, 0.22941421, 0.2450912]

# Определение преобразований
transforms = v2.Compose([
    v2.ToTensor(),
    v2.Normalize(mean=mean, std=std),
])

# Создание датасетов
train_dataset = ImageFolder(root="./data/train", transform=transforms)
test_dataset = ImageFolder(root="./data/test", transform=transforms)

# Получение информации
num_train_images = len(train_dataset)
num_test_images = len(test_dataset)
num_classes = len(train_dataset.classes)
# Получение случайного изображения из обучающего набора для определения размеров и количества каналов
image, _ = train_dataset[0]
channels, height, width = image.shape
print(f'Количество изображений в обучающей выборке: {num_train_images}')
print(f'Количество изображений в тестовой выборке: {num_test_images}')
print(f'Количество классов: {num_classes}')
print(f'Количество каналов в изображении: {channels}')
print(f'Высота и ширина изображения: {height}x{width}')

Количество изображений в обучающей выборке: 8000
Количество изображений в тестовой выборке: 2000
Количество классов: 10
Количество каналов в изображении: 3
Высота и ширина изображения: 224x224


## Создание DataLoader

Далее необходимо подготовить три загрузчика данных:

1. Обучающий
2. Проверочный
3. Тестовый

Тестовый загрузчик делается из тестового Dataset, а обучающий и проверочный необходимо создать, используя [SubsetRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.SubsetRandomSampler), для его работы требуется массив индексов, по которым в дальнейшем загрузчик будет в случайном порядке брать изображения и лейблы.

In [33]:
# Импорт необходимых библиотек
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

# Определение размера валидационного набора как 20% от общего количества образцов
val_size = 0.2

# Определение общего числа образцов в обучающем наборе данных
train_samples = len(train_dataset)
# Создание списка индексов для всех образцов в обучающем наборе
train_indices = list(range(train_samples))

# Вычисление индекса, который будет использоваться для разделения на обучающий и валидационный наборы
split_value = int(np.floor(val_size * train_samples))
# Перемешивание индексов для обеспечения случайности разделения
np.random.shuffle(train_indices)

# Разделение индексов на обучающие и валидационные, используя вычисленный индекс разделения
# Обучающие индексы получаются путем выбора элементов после индекса разделения
train_idx, val_idx = train_indices[split_value:], train_indices[:split_value]

# Создание объектов Sampler для обучающего и валидационного наборов
# Эти объекты будут случайным образом выбирать элементы из соответствующих индексов
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

**Опишите своими словами, что делает каждая строчка кода в предыдущей ячейке.**

Ваш ответ: Расписано в коментариях

In [18]:
from torch.utils.data import DataLoader

# Определяем размер батча (количество образцов данных, обрабатываемых за одну итерацию)
BATCH_SIZE = 32

# Создаём DataLoader для обучающего набора данных.
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
# Создаём DataLoader для валидационного набора данных.
val_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)
# Создаём DataLoader для валидационного набора данных.
test_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


**Какую задачу решает Dataloader?**

Ваш ответ:
1. Автоматизация батчинга: DataLoader автоматически группирует данные в батчи заданного размера, что важно для эффективного обучения моделей глубокого обучения, поскольку обработка данных пакетами помогает оптимизировать вычислительные ресурсы.

2. Перемешивание данных: В случае обучающего набора данных DataLoader может перемешивать данные перед формированием батчей. Это предотвращает модель от запоминания порядка следования данных и способствует лучшему обобщению.

3. Параллельная загрузка и предобработка: DataLoader поддерживает многопоточную загрузку данных, что снижает время ожидания ввода данных и ускоряет обучение. Предобработка данных (например, нормализация, аугментация) также может быть интегрирована и выполнена параллельно.

**Почему использование трех выборок (обучающей, валидационной, тестовой) считается хорошей практикой?**

Ваш ответ:
1. Обучающая выборка: Используется непосредственно для обучения модели. Это основной набор данных, на котором модель "учится" на примерах, адаптируя свои веса для минимизации ошибки.

2. Валидационная выборка: Используется для оценки модели в процессе обучения, но не для обучения. Основная цель валидационной выборки — помочь в настройке гиперпараметров модели и предотвратить переобучение. Модель не "видит" данные из валидационной выборки в процессе обучения, что позволяет проверить, насколько хорошо модель обобщает на новые данные.

3. Тестовая выборка: Используется после завершения процесса обучения для окончательной оценки модели. Тестовая выборка позволяет проверить, насколько эффективно модель работает на данных, которые никогда не использовались в процессе обучения или настройки. Это конечная проверка обобщающей способности модели.

## Создание модели

За основу можно взять модель LeNet-5, но скорее всего вам придется ее адаптировать под свою задачу, так как в большинстве случаев она написана под черно-белые изображения размером 32 на 32 пикселя.

Сверточные нейронные сети состоят из двух частей:
1. Слои свертки(функции свертки, активации, субдискретизации)
2. Полносвязные слои (MLP)

Слои можно объединить с помощью `nn.Sequential()`. А класс вашей модели должен наследоваться от `nn.Module`.

`def forward()` определяет прямой ход и должна возвращать итоговый результат работы модели - в данном случае логиты.

In [34]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        # Определение первого свёрточного слоя
        # 3 входных канала (для RGB изображений), 16 выходных каналов, ядро размером 5,
        # шаг свёртки 1 и паддинг 2 для сохранения размерности изображения.
        # После свёртки применяется функция активации ReLU и операция максимального пулинга с ядром 2 и шагом 2.
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        # Определение второго свёрточного слоя, аналогичного первому,
        # но с 32 выходными каналами.
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        # Определение полносвязных слоёв:
        # fc1 преобразует данные из двумерного формата в вектор, fc2 и fc3 далее уменьшают размерность,
        # ведущую к выходному слою, который соответствует количеству классов.
        # Важно отметить, что размер входа в fc1 должен соответствовать размеру данных после свёрточных и пулинг слоёв.
        self.fc1 = nn.Linear(32 * 56 * 56, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    # Определение прямого прохода через сеть
    def forward(self, x):
        out = self.layer1(x) # Проход через первый свёрточный слой
        out = self.layer2(out) # Проход через второй свёрточный слой
        
        out = out.view(out.size(0), -1) # Преобразование данных для полносвязного слоя
        
        out = F.relu(self.fc1(out)) # Проход через первый полносвязный слой с активацией ReLU
        out = F.relu(self.fc2(out)) # Проход через второй полносвязный слой с активацией ReLU
        out = self.fc3(out) # Выход через последний полносвязный слой без активации
        return out


**Опишите суть операции свертки.**

Ваш ответ: 
- Операция свёртки в контексте обработки изображений и глубокого обучения представляет собой математическую процедуру, при которой изображение "просматривается" через так называемое свёрточное ядро (или фильтр) для получения новой матрицы, называемой картой признаков. Это ядро имеет меньшие размеры по сравнению с исходным изображением и "скользит" по всему изображению, вычисляя в каждом положении сумму произведений его значений и соответствующих значений изображения под ним.

- Процесс свёртки помогает выявить определённые характеристики изображения, такие как границы, углы или текстуры, в зависимости от типа применяемого фильтра. При обучении нейронной сети параметры этих фильтров оптимизируются таким образом, чтобы лучше всего выделять полезные для задачи признаки.

**Опишите суть операции субдискретизации.**

Ваш ответ: Операция субдискретизации, также известная как пулинг, является процессом уменьшения размерности полученной карты признаков, сохраняя при этом важные информационные сигналы.



## Создание объекта модели, функции потерь и оптимизатора

В качестве функции потерь будет использована перекрестная энтропия, в задании MLP вы фактически ее реализовали, но через набор отдельных функций.

В качестве оптимизатора можете взять стохастический градиентный спуск или Adam.

In [35]:
import torch.optim as optim

# Создание экземпляра модели
model = ConvNet(num_classes=10)

# Функция потерь
loss_fn = nn.CrossEntropyLoss()

# Оптимизатор
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

## Цикл обучения

Попробуйте разные гиперпараметры для вашей модели. Попробуйте улучшить результат от первой попытки.

Вы можете по своему желанию добавить графики потерь и точности от эпохи.

In [36]:
num_epochs = 10 #кол-во эпох
for epoch in range(num_epochs):
    model.train()  # Переключаем модель в режим обучения
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        # Прямой ход
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Обнуление градиентов
        optimizer.zero_grad()

        # Обратный ход
        loss.backward()

        # Шаг оптимизатора
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # Валидация
    model.eval()  # Переключаем модель в режим валидации
    correct = 0
    total = 0
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the validation images: {accuracy:.2f} %')


Epoch [1/10], Loss: 1.9978
Accuracy of the model on the validation images: 44.31 %
Epoch [2/10], Loss: 1.5519
Accuracy of the model on the validation images: 50.94 %
Epoch [3/10], Loss: 1.3807
Accuracy of the model on the validation images: 56.25 %
Epoch [4/10], Loss: 1.2333
Accuracy of the model on the validation images: 57.06 %
Epoch [5/10], Loss: 1.1660
Accuracy of the model on the validation images: 60.50 %
Epoch [6/10], Loss: 1.0635
Accuracy of the model on the validation images: 58.00 %
Epoch [7/10], Loss: 1.0012
Accuracy of the model on the validation images: 62.19 %
Epoch [8/10], Loss: 0.9349
Accuracy of the model on the validation images: 63.94 %
Epoch [9/10], Loss: 0.8875
Accuracy of the model on the validation images: 63.62 %
Epoch [10/10], Loss: 0.8531
Accuracy of the model on the validation images: 65.94 %


## Итоговая оценка

Часть кода, которую вы реализовали для оценки модели на валидационной выборке, можно использовать для финальной проверки, указав нужный loader.

In [37]:
with torch.no_grad():
    model.eval()  # Переключение модели в режим оценки
    correct = 0
    total = 0
    for images, labels in test_loader:  # Использование DataLoader тестового набора
        outputs = model(images)  # Расчет вывода модели для данного батча
        _, predicted = torch.max(outputs.data, 1)  # Получение индексов максимальных значений для предсказаний
        total += labels.size(0)  # Подсчет общего количества меток
        correct += (predicted == labels).sum().item()  # Подсчет количества правильных предсказаний

    print(f'Точность (accuracy) на тестовом наборе данных: {100 * correct / total}%')

Точность (accuracy) на тестовом наборе данных: 66.45%


**Точность работы модели на тестовой выборке**

Ваш ответ: 66.45%